In [13]:
import numpy as np
import pandas as pd
import csv

from scipy.spatial.distance import cdist

from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.algorithms.moo.nsga2 import NSGA2


from pymoo.operators.selection.tournament import TournamentSelection
from pymoo.algorithms.moo.nsga2 import binary_tournament
from pymoo.algorithms.moo.nsga3 import comp_by_cv_then_random

from pymoo.core.problem import ElementwiseProblem

from pymoo.operators.sampling.rnd import PermutationRandomSampling
from pymoo.operators.crossover.ox import OrderCrossover
from pymoo.operators.mutation.inversion import InversionMutation

from pymoo.util.display.column import Column
from pymoo.util.display.output import Output
from pymoo.util.display.display import Display

from pymoo.optimize import minimize
from pymoo.util.ref_dirs import get_reference_directions

import csv
import time

In [4]:
kroA100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroA100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")
kroB100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroB100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")
kroC100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroC100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")

In [15]:
class measure_time(Output):
    def __init__(self):
        super().__init__()
        self.x_time = Column("x_time", width=13)
        self.columns += [self.x_time]

    def update(self, algorithm):
        super().update(algorithm)
        writer.writerow([time.time() - start_time])
        self.x_time.set(time.time() - start_time)

In [5]:
class TravelingSalesman(ElementwiseProblem):
    def __init__(self, cost_one, cost_two, cost_three, **kwargs):
        n_one, _   = cost_one.shape

        self.one = cost_one
        self.two = cost_two
        self.three = cost_three
        
        super().__init__(n_var=n_one, n_obj=3, xl=0, xu=n_one, vtype=int, **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
        out['F'] = self.get_route_length(x)

    def get_route_length(self, x):
        n_cities = len(x)
        dist_one = 0
        dist_two = 0
        dist_three = 0
        
        for k in range(n_cities - 1):
            i, j = x[k], x[k + 1]
            dist_one += self.one[i, j]
            dist_two += self.two[i, j]
            dist_three += self.three[i, j]

        last, first = x[-1], x[0]
        
        dist_one += self.one[last, first]
        dist_one = 100000/dist_one
        dist_two += self.two[last, first]
        dist_two = 100000/dist_two
        dist_three += self.three[last, first]
        dist_three = 100000/dist_three
        
        return dist_one, dist_two, dist_three

In [6]:
cost_one = kroA100.values
cost_two = kroB100.values
cost_three = kroC100.values

one = cdist(cost_one,cost_one)
two = cdist(cost_two,cost_two)
three = cdist(cost_three,cost_three)

In [7]:
problem = TravelingSalesman(cost_one=one, cost_two=two, cost_three=three)

In [8]:
ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=12)

In [9]:
algorithm_nsga3 = NSGA3(pop_size=91,
                        sampling=PermutationRandomSampling(),
                        selection=TournamentSelection(func_comp=comp_by_cv_then_random),
                        crossover=OrderCrossover(),
                        mutation=InversionMutation(),
                        ref_dirs=ref_dirs,
                        eliminate_duplicates=False)

In [10]:
algorithm_nsga2 = NSGA2(pop_size=92,
                        sampling=PermutationRandomSampling(),
                        selection=TournamentSelection(func_comp=binary_tournament),
                        crossover=OrderCrossover(),
                        mutation=InversionMutation(),
                        eliminate_duplicates=False)

In [23]:
with open('pymoo_time_nsga2_motsp_92_500_3_100.csv', 'w', newline='') as f, open('pymoo_fitness_nsga2_motsp_92_500_3_100.csv','w', newline='') as g:
    writer = csv.writer(f)
    writer.writerow(["Execution Time"])
    
    writerFitness = csv.writer(g)
    writer.writerow(["f1","f2","f3"])
    start_time = time.time()
    res_motsp_nsga2 = minimize(problem, algorithm_nsga2, ('n_gen', 5000), output=measure_time(), verbose=True, save_history=False, seed=1)    
    for fitness in res_motsp_nsga2.F:
            writerFitness.writerow(fitness)

n_gen  |  n_eval  |     x_time   
     1 |       92 |  0.0168905258
     2 |      184 |  0.0548055172
     3 |      276 |  0.0728421211
     4 |      368 |  0.1041290760
     5 |      460 |  0.1197497845
     6 |      552 |  0.1509919167
     7 |      644 |  0.1731288433
     8 |      736 |  0.2044184208
     9 |      828 |  0.2351088524
    10 |      920 |  0.2774319649
    11 |     1012 |  0.2991576195
    12 |     1104 |  0.3147821426
    13 |     1196 |  0.3460230827
    14 |     1288 |  0.3736443520
    15 |     1380 |  0.3893167973
    16 |     1472 |  0.4205522537
    17 |     1564 |  0.4517955780
    18 |     1656 |  0.4862723351
    19 |     1748 |  0.5012586117
    20 |     1840 |  0.5325467587
    21 |     1932 |  0.5481235981
    22 |     2024 |  0.5742938519
    23 |     2116 |  0.6055407524
    24 |     2208 |  0.6212050915
    25 |     2300 |  0.6524448395
    26 |     2392 |  0.6897916794
    27 |     2484 |  0.7168192863
    28 |     2576 |  0.7335121632
    29 |     2

   242 |    22264 |  6.5807540417
   243 |    22356 |  6.6020176411
   244 |    22448 |  6.6352093220
   245 |    22540 |  6.6681029797
   246 |    22632 |  6.6870510578
   247 |    22724 |  6.7182986736
   248 |    22816 |  6.7339191437
   249 |    22908 |  6.7652053833
   250 |    23000 |  6.7873799801
   251 |    23092 |  6.8187625408
   252 |    23184 |  6.8564932346
   253 |    23276 |  6.8658506870
   254 |    23368 |  6.9067108631
   255 |    23460 |  6.9335424900
   256 |    23552 |  6.9668717384
   257 |    23644 |  6.9918026924
   258 |    23736 |  7.0148148537
   259 |    23828 |  7.0304362774
   260 |    23920 |  7.0689194202
   261 |    24012 |  7.1041600704
   262 |    24104 |  7.1290540695
   263 |    24196 |  7.1481363773
   264 |    24288 |  7.1792547703
   265 |    24380 |  7.2015857697
   266 |    24472 |  7.2349560261
   267 |    24564 |  7.2709434032
   268 |    24656 |  7.3071408272
   269 |    24748 |  7.3325157166
   270 |    24840 |  7.3481373787
   271 |    24

   483 |    44436 |  1.352212E+01
   484 |    44528 |  1.355112E+01
   485 |    44620 |  1.357566E+01
   486 |    44712 |  1.360319E+01
   487 |    44804 |  1.362798E+01
   488 |    44896 |  1.365385E+01
   489 |    44988 |  1.367957E+01
   490 |    45080 |  1.370627E+01
   491 |    45172 |  1.373541E+01
   492 |    45264 |  1.376750E+01
   493 |    45356 |  1.379446E+01
   494 |    45448 |  1.381883E+01
   495 |    45540 |  1.384470E+01
   496 |    45632 |  1.386951E+01
   497 |    45724 |  1.389355E+01
   498 |    45816 |  1.392154E+01
   499 |    45908 |  1.394999E+01
   500 |    46000 |  1.398517E+01
   501 |    46092 |  1.401010E+01
   502 |    46184 |  1.403150E+01
   503 |    46276 |  1.405555E+01
   504 |    46368 |  1.408357E+01
   505 |    46460 |  1.410755E+01
   506 |    46552 |  1.413320E+01
   507 |    46644 |  1.415689E+01
   508 |    46736 |  1.419240E+01
   509 |    46828 |  1.421878E+01
   510 |    46920 |  1.424416E+01
   511 |    47012 |  1.426816E+01
   512 |    47

   730 |    67160 |  2.032513E+01
   731 |    67252 |  2.035231E+01
   732 |    67344 |  2.037827E+01
   733 |    67436 |  2.040638E+01
   734 |    67528 |  2.043191E+01
   735 |    67620 |  2.045834E+01
   736 |    67712 |  2.048490E+01
   737 |    67804 |  2.052334E+01
   738 |    67896 |  2.055135E+01
   739 |    67988 |  2.058316E+01
   740 |    68080 |  2.060819E+01
   741 |    68172 |  2.063219E+01
   742 |    68264 |  2.065622E+01
   743 |    68356 |  2.068422E+01
   744 |    68448 |  2.072022E+01
   745 |    68540 |  2.075622E+01
   746 |    68632 |  2.079148E+01
   747 |    68724 |  2.082081E+01
   748 |    68816 |  2.084481E+01
   749 |    68908 |  2.088081E+01
   750 |    69000 |  2.090481E+01
   751 |    69092 |  2.093681E+01
   752 |    69184 |  2.096081E+01
   753 |    69276 |  2.098481E+01
   754 |    69368 |  2.101281E+01
   755 |    69460 |  2.104176E+01
   756 |    69552 |  2.106576E+01
   757 |    69644 |  2.108683E+01
   758 |    69736 |  2.111483E+01
   759 |    69

   975 |    89700 |  2.702084E+01
   976 |    89792 |  2.704677E+01
   977 |    89884 |  2.707289E+01
   978 |    89976 |  2.709689E+01
   979 |    90068 |  2.712089E+01
   980 |    90160 |  2.714485E+01
   981 |    90252 |  2.716889E+01
   982 |    90344 |  2.719289E+01
   983 |    90436 |  2.721690E+01
   984 |    90528 |  2.724085E+01
   985 |    90620 |  2.727466E+01
   986 |    90712 |  2.729879E+01
   987 |    90804 |  2.732090E+01
   988 |    90896 |  2.734491E+01
   989 |    90988 |  2.736890E+01
   990 |    91080 |  2.739687E+01
   991 |    91172 |  2.742086E+01
   992 |    91264 |  2.744886E+01
   993 |    91356 |  2.748406E+01
   994 |    91448 |  2.750947E+01
   995 |    91540 |  2.753252E+01
   996 |    91632 |  2.755652E+01
   997 |    91724 |  2.758448E+01
   998 |    91816 |  2.760848E+01
   999 |    91908 |  2.763648E+01
  1000 |    92000 |  2.766048E+01
  1001 |    92092 |  2.768848E+01
  1002 |    92184 |  2.771877E+01
  1003 |    92276 |  2.774872E+01
  1004 |    92

...

  3908 |   359536 |  1.149179E+02
  3909 |   359628 |  1.149819E+02
  3910 |   359720 |  1.150619E+02
  3911 |   359812 |  1.151819E+02
  3912 |   359904 |  1.152539E+02
  3913 |   359996 |  1.152961E+02
  3914 |   360088 |  1.153235E+02
  3915 |   360180 |  1.153675E+02
  3916 |   360272 |  1.154155E+02
  3917 |   360364 |  1.154675E+02
  3918 |   360456 |  1.155195E+02
  3919 |   360548 |  1.155555E+02
  3920 |   360640 |  1.155915E+02
  3921 |   360732 |  1.156475E+02
  3922 |   360824 |  1.157155E+02
  3923 |   360916 |  1.157875E+02
  3924 |   361008 |  1.158515E+02
  3925 |   361100 |  1.159115E+02
  3926 |   361192 |  1.159595E+02
  3927 |   361284 |  1.160635E+02
  3928 |   361376 |  1.161355E+02
  3929 |   361468 |  1.161995E+02
  3930 |   361560 |  1.162915E+02
  3931 |   361652 |  1.163795E+02
  3932 |   361744 |  1.164635E+02
  3933 |   361836 |  1.165475E+02
  3934 |   361928 |  1.166185E+02
  3935 |   362020 |  1.167252E+02
  3936 |   362112 |  1.168120E+02
  3937 |   362

  4156 |   382352 |  1.244411E+02
  4157 |   382444 |  1.244680E+02
  4158 |   382536 |  1.244979E+02
  4159 |   382628 |  1.245229E+02
  4160 |   382720 |  1.245518E+02
  4161 |   382812 |  1.245767E+02
  4162 |   382904 |  1.246026E+02
  4163 |   382996 |  1.246325E+02
  4164 |   383088 |  1.246634E+02
  4165 |   383180 |  1.246894E+02
  4166 |   383272 |  1.247163E+02
  4167 |   383364 |  1.247492E+02
  4168 |   383456 |  1.247781E+02
  4169 |   383548 |  1.248051E+02
  4170 |   383640 |  1.248300E+02
  4171 |   383732 |  1.248560E+02
  4172 |   383824 |  1.248868E+02
  4173 |   383916 |  1.249138E+02
  4174 |   384008 |  1.249507E+02
  4175 |   384100 |  1.249995E+02
  4176 |   384192 |  1.250344E+02
  4177 |   384284 |  1.250703E+02
  4178 |   384376 |  1.251182E+02
  4179 |   384468 |  1.251551E+02
  4180 |   384560 |  1.251840E+02
  4181 |   384652 |  1.252160E+02
  4182 |   384744 |  1.252419E+02
  4183 |   384836 |  1.252718E+02
  4184 |   384928 |  1.252997E+02
  4185 |   385

  4398 |   404616 |  1.351243E+02
  4399 |   404708 |  1.351643E+02
  4400 |   404800 |  1.351963E+02
  4401 |   404892 |  1.352323E+02
  4402 |   404984 |  1.352643E+02
  4403 |   405076 |  1.353003E+02
  4404 |   405168 |  1.353363E+02
  4405 |   405260 |  1.353643E+02
  4406 |   405352 |  1.353923E+02
  4407 |   405444 |  1.354243E+02
  4408 |   405536 |  1.354523E+02
  4409 |   405628 |  1.354803E+02
  4410 |   405720 |  1.355043E+02
  4411 |   405812 |  1.355323E+02
  4412 |   405904 |  1.355683E+02
  4413 |   405996 |  1.356031E+02
  4414 |   406088 |  1.356311E+02
  4415 |   406180 |  1.356631E+02
  4416 |   406272 |  1.356911E+02
  4417 |   406364 |  1.357231E+02
  4418 |   406456 |  1.357511E+02
  4419 |   406548 |  1.357871E+02
  4420 |   406640 |  1.358336E+02
  4421 |   406732 |  1.358696E+02
  4422 |   406824 |  1.359136E+02
  4423 |   406916 |  1.359456E+02
  4424 |   407008 |  1.359776E+02
  4425 |   407100 |  1.360016E+02
  4426 |   407192 |  1.360336E+02
  4427 |   407

  4646 |   427432 |  1.436719E+02
  4647 |   427524 |  1.437041E+02
  4648 |   427616 |  1.437287E+02
  4649 |   427708 |  1.437708E+02
  4650 |   427800 |  1.438022E+02
  4651 |   427892 |  1.438348E+02
  4652 |   427984 |  1.438724E+02
  4653 |   428076 |  1.439101E+02
  4654 |   428168 |  1.439373E+02
  4655 |   428260 |  1.439758E+02
  4656 |   428352 |  1.440065E+02
  4657 |   428444 |  1.440367E+02
  4658 |   428536 |  1.440658E+02
  4659 |   428628 |  1.440987E+02
  4660 |   428720 |  1.441306E+02
  4661 |   428812 |  1.441765E+02
  4662 |   428904 |  1.442121E+02
  4663 |   428996 |  1.442460E+02
  4664 |   429088 |  1.442799E+02
  4665 |   429180 |  1.443101E+02
  4666 |   429272 |  1.443370E+02
  4667 |   429364 |  1.443663E+02
  4668 |   429456 |  1.443920E+02
  4669 |   429548 |  1.444160E+02
  4670 |   429640 |  1.444400E+02
  4671 |   429732 |  1.444640E+02
  4672 |   429824 |  1.444920E+02
  4673 |   429916 |  1.445160E+02
  4674 |   430008 |  1.445400E+02
  4675 |   430

  4888 |   449696 |  1.510779E+02
  4889 |   449788 |  1.511158E+02
  4890 |   449880 |  1.511421E+02
  4891 |   449972 |  1.511661E+02
  4892 |   450064 |  1.511981E+02
  4893 |   450156 |  1.512222E+02
  4894 |   450248 |  1.512462E+02
  4895 |   450340 |  1.512742E+02
  4896 |   450432 |  1.513192E+02
  4897 |   450524 |  1.513464E+02
  4898 |   450616 |  1.513764E+02
  4899 |   450708 |  1.514084E+02
  4900 |   450800 |  1.514324E+02
  4901 |   450892 |  1.514604E+02
  4902 |   450984 |  1.514845E+02
  4903 |   451076 |  1.515084E+02
  4904 |   451168 |  1.515324E+02
  4905 |   451260 |  1.515830E+02
  4906 |   451352 |  1.516128E+02
  4907 |   451444 |  1.516408E+02
  4908 |   451536 |  1.516688E+02
  4909 |   451628 |  1.517008E+02
  4910 |   451720 |  1.517307E+02
  4911 |   451812 |  1.517581E+02
  4912 |   451904 |  1.518018E+02
  4913 |   451996 |  1.518259E+02
  4914 |   452088 |  1.518498E+02
  4915 |   452180 |  1.518778E+02
  4916 |   452272 |  1.519019E+02
  4917 |   452

In [24]:
with open('pymoo_time_nsga3_motsp_92_500_3_100.csv', 'w', newline='') as f, open('pymoo_fitness_nsga3_motsp_92_500_3_100.csv','w', newline='') as g:
    writer = csv.writer(f)
    writer.writerow(["Execution Time"])
    
    writerFitness = csv.writer(g)
    writer.writerow(["f1","f2","f3"])
    start_time = time.time()
    res_motsp_nsga3 = minimize(problem, algorithm_nsga3, ('n_gen', 5000), output=measure_time(), verbose=True, save_history=False, seed=1)
    for fitness in res_motsp_nsga3.F:
            writerFitness.writerow(fitness)

n_gen  |  n_eval  |     x_time   
     1 |       91 |  0.0299353600
     2 |      182 |  0.0591166019
     3 |      273 |  0.0901417732
     4 |      364 |  0.1341416836
     5 |      455 |  0.1661450863
     6 |      546 |  0.1981441975
     7 |      637 |  0.2341461182
     8 |      728 |  0.2758846283
     9 |      819 |  0.3278272152
    10 |      910 |  0.3798274994
    11 |     1001 |  0.4158267975
    12 |     1092 |  0.4518260956
    13 |     1183 |  0.5070395470
    14 |     1274 |  0.5669238567
    15 |     1365 |  0.5992197990
    16 |     1456 |  0.6472189426
    17 |     1547 |  0.6792609692
    18 |     1638 |  0.7245407104
    19 |     1729 |  0.7646832466
    20 |     1820 |  0.7966790199
    21 |     1911 |  0.8246779442
    22 |     2002 |  0.8606781960
    23 |     2093 |  0.9006805420
    24 |     2184 |  0.9353024960
    25 |     2275 |  0.9745056629
    26 |     2366 |  1.0026364326
    27 |     2457 |  1.0319576263
    28 |     2548 |  1.0597479343
    29 |     2

   245 |    22295 |  8.2068417072
   246 |    22386 |  8.2348368168
   247 |    22477 |  8.2668371201
   248 |    22568 |  8.2947986126
   249 |    22659 |  8.3228402138
   250 |    22750 |  8.3548419476
   251 |    22841 |  8.3908004761
   252 |    22932 |  8.4187974930
   253 |    23023 |  8.4507999420
   254 |    23114 |  8.4788365364
   255 |    23205 |  8.5068674088
   256 |    23296 |  8.5348682404
   257 |    23387 |  8.5668017864
   258 |    23478 |  8.5948364735
   259 |    23569 |  8.6267979145
   260 |    23660 |  8.6668000221
   261 |    23751 |  8.6988651752
   262 |    23842 |  8.7347981930
   263 |    23933 |  8.7908005714
   264 |    24024 |  8.8267989159
   265 |    24115 |  8.8587977886
   266 |    24206 |  8.8947982788
   267 |    24297 |  8.9348075390
   268 |    24388 |  8.9628357887
   269 |    24479 |  8.9948351383
   270 |    24570 |  9.0348341465
   271 |    24661 |  9.0707967281
   272 |    24752 |  9.1267967224
   273 |    24843 |  9.1628346443
   274 |    24

   489 |    44499 |  1.718765E+01
   490 |    44590 |  1.726643E+01
   491 |    44681 |  1.733525E+01
   492 |    44772 |  1.738711E+01
   493 |    44863 |  1.744107E+01
   494 |    44954 |  1.748695E+01
   495 |    45045 |  1.754978E+01
   496 |    45136 |  1.759965E+01
   497 |    45227 |  1.768427E+01
   498 |    45318 |  1.775627E+01
   499 |    45409 |  1.788427E+01
   500 |    45500 |  1.796427E+01
   501 |    45591 |  1.803834E+01
   502 |    45682 |  1.810495E+01
   503 |    45773 |  1.818895E+01
   504 |    45864 |  1.828895E+01
   505 |    45955 |  1.839695E+01
   506 |    46046 |  1.846895E+01
   507 |    46137 |  1.852388E+01
   508 |    46228 |  1.857409E+01
   509 |    46319 |  1.871008E+01
   510 |    46410 |  1.881408E+01
   511 |    46501 |  1.889408E+01
   512 |    46592 |  1.901408E+01
   513 |    46683 |  1.915008E+01
   514 |    46774 |  1.927408E+01
   515 |    46865 |  1.938208E+01
   516 |    46956 |  1.943664E+01
   517 |    47047 |  1.948464E+01
   518 |    47

   731 |    66521 |  2.677856E+01
   732 |    66612 |  2.681460E+01
   733 |    66703 |  2.684660E+01
   734 |    66794 |  2.687856E+01
   735 |    66885 |  2.690663E+01
   736 |    66976 |  2.693856E+01
   737 |    67067 |  2.697460E+01
   738 |    67158 |  2.700656E+01
   739 |    67249 |  2.703460E+01
   740 |    67340 |  2.707460E+01
   741 |    67431 |  2.710656E+01
   742 |    67522 |  2.713460E+01
   743 |    67613 |  2.717060E+01
   744 |    67704 |  2.720262E+01
   745 |    67795 |  2.723456E+01
   746 |    67886 |  2.726656E+01
   747 |    67977 |  2.729860E+01
   748 |    68068 |  2.733856E+01
   749 |    68159 |  2.744256E+01
   750 |    68250 |  2.752656E+01
   751 |    68341 |  2.763323E+01
   752 |    68432 |  2.772929E+01
   753 |    68523 |  2.779328E+01
   754 |    68614 |  2.788528E+01
   755 |    68705 |  2.797728E+01
   756 |    68796 |  2.804172E+01
   757 |    68887 |  2.809772E+01
   758 |    68978 |  2.815372E+01
   759 |    69069 |  2.820972E+01
   760 |    69

   974 |    88634 |  3.682158E+01
   975 |    88725 |  3.684884E+01
   976 |    88816 |  3.688084E+01
   977 |    88907 |  3.691284E+01
   978 |    88998 |  3.694484E+01
   979 |    89089 |  3.697684E+01
   980 |    89180 |  3.700884E+01
   981 |    89271 |  3.704084E+01
   982 |    89362 |  3.706888E+01
   983 |    89453 |  3.710084E+01
   984 |    89544 |  3.713284E+01
   985 |    89635 |  3.716084E+01
   986 |    89726 |  3.719284E+01
   987 |    89817 |  3.722487E+01
   988 |    89908 |  3.725684E+01
   989 |    89999 |  3.728484E+01
   990 |    90090 |  3.732084E+01
   991 |    90181 |  3.735684E+01
   992 |    90272 |  3.738884E+01
   993 |    90363 |  3.741684E+01
   994 |    90454 |  3.744884E+01
   995 |    90545 |  3.747684E+01
   996 |    90636 |  3.750884E+01
   997 |    90727 |  3.753684E+01
   998 |    90818 |  3.756884E+01
   999 |    90909 |  3.760087E+01
  1000 |    91000 |  3.762884E+01
  1001 |    91091 |  3.766484E+01
  1002 |    91182 |  3.769687E+01
  1003 |    91

...

  3902 |   355082 |  1.402585E+02
  3903 |   355173 |  1.402905E+02
  3904 |   355264 |  1.403266E+02
  3905 |   355355 |  1.403625E+02
  3906 |   355446 |  1.403985E+02
  3907 |   355537 |  1.404385E+02
  3908 |   355628 |  1.404745E+02
  3909 |   355719 |  1.405066E+02
  3910 |   355810 |  1.405545E+02
  3911 |   355901 |  1.406105E+02
  3912 |   355992 |  1.406425E+02
  3913 |   356083 |  1.406785E+02
  3914 |   356174 |  1.407665E+02
  3915 |   356265 |  1.408145E+02
  3916 |   356356 |  1.408625E+02
  3917 |   356447 |  1.408985E+02
  3918 |   356538 |  1.409345E+02
  3919 |   356629 |  1.409665E+02
  3920 |   356720 |  1.410025E+02
  3921 |   356811 |  1.410305E+02
  3922 |   356902 |  1.410585E+02
  3923 |   356993 |  1.410905E+02
  3924 |   357084 |  1.411225E+02
  3925 |   357175 |  1.411545E+02
  3926 |   357266 |  1.411785E+02
  3927 |   357357 |  1.412106E+02
  3928 |   357448 |  1.412385E+02
  3929 |   357539 |  1.412706E+02
  3930 |   357630 |  1.412986E+02
  3931 |   357

  4145 |   377195 |  1.481203E+02
  4146 |   377286 |  1.481582E+02
  4147 |   377377 |  1.481951E+02
  4148 |   377468 |  1.482310E+02
  4149 |   377559 |  1.482699E+02
  4150 |   377650 |  1.482998E+02
  4151 |   377741 |  1.483278E+02
  4152 |   377832 |  1.483567E+02
  4153 |   377923 |  1.483906E+02
  4154 |   378014 |  1.484255E+02
  4155 |   378105 |  1.484525E+02
  4156 |   378196 |  1.484804E+02
  4157 |   378287 |  1.485073E+02
  4158 |   378378 |  1.485332E+02
  4159 |   378469 |  1.485642E+02
  4160 |   378560 |  1.485930E+02
  4161 |   378651 |  1.486220E+02
  4162 |   378742 |  1.486499E+02
  4163 |   378833 |  1.486778E+02
  4164 |   378924 |  1.487058E+02
  4165 |   379015 |  1.487327E+02
  4166 |   379106 |  1.487636E+02
  4167 |   379197 |  1.487916E+02
  4168 |   379288 |  1.488195E+02
  4169 |   379379 |  1.488484E+02
  4170 |   379470 |  1.488833E+02
  4171 |   379561 |  1.489112E+02
  4172 |   379652 |  1.489382E+02
  4173 |   379743 |  1.489700E+02
  4174 |   379

  4388 |   399308 |  1.554266E+02
  4389 |   399399 |  1.554585E+02
  4390 |   399490 |  1.554964E+02
  4391 |   399581 |  1.555253E+02
  4392 |   399672 |  1.555533E+02
  4393 |   399763 |  1.555802E+02
  4394 |   399854 |  1.556091E+02
  4395 |   399945 |  1.556420E+02
  4396 |   400036 |  1.556750E+02
  4397 |   400127 |  1.557029E+02
  4398 |   400218 |  1.557299E+02
  4399 |   400309 |  1.557588E+02
  4400 |   400400 |  1.557867E+02
  4401 |   400491 |  1.558207E+02
  4402 |   400582 |  1.558495E+02
  4403 |   400673 |  1.558775E+02
  4404 |   400764 |  1.559114E+02
  4405 |   400855 |  1.559403E+02
  4406 |   400946 |  1.559673E+02
  4407 |   401037 |  1.559952E+02
  4408 |   401128 |  1.560221E+02
  4409 |   401219 |  1.560500E+02
  4410 |   401310 |  1.560790E+02
  4411 |   401401 |  1.561069E+02
  4412 |   401492 |  1.561378E+02
  4413 |   401583 |  1.561687E+02
  4414 |   401674 |  1.561967E+02
  4415 |   401765 |  1.562246E+02
  4416 |   401856 |  1.562515E+02
  4417 |   401

  4631 |   421421 |  1.651623E+02
  4632 |   421512 |  1.652223E+02
  4633 |   421603 |  1.652543E+02
  4634 |   421694 |  1.652943E+02
  4635 |   421785 |  1.653263E+02
  4636 |   421876 |  1.653583E+02
  4637 |   421967 |  1.653903E+02
  4638 |   422058 |  1.654223E+02
  4639 |   422149 |  1.654543E+02
  4640 |   422240 |  1.654782E+02
  4641 |   422331 |  1.655183E+02
  4642 |   422422 |  1.655463E+02
  4643 |   422513 |  1.655783E+02
  4644 |   422604 |  1.656103E+02
  4645 |   422695 |  1.656463E+02
  4646 |   422786 |  1.656743E+02
  4647 |   422877 |  1.657063E+02
  4648 |   422968 |  1.657383E+02
  4649 |   423059 |  1.657703E+02
  4650 |   423150 |  1.658063E+02
  4651 |   423241 |  1.658423E+02
  4652 |   423332 |  1.658743E+02
  4653 |   423423 |  1.659063E+02
  4654 |   423514 |  1.659623E+02
  4655 |   423605 |  1.660063E+02
  4656 |   423696 |  1.660543E+02
  4657 |   423787 |  1.660943E+02
  4658 |   423878 |  1.661303E+02
  4659 |   423969 |  1.661663E+02
  4660 |   424

  4873 |   443443 |  1.753296E+02
  4874 |   443534 |  1.753656E+02
  4875 |   443625 |  1.754056E+02
  4876 |   443716 |  1.754376E+02
  4877 |   443807 |  1.754736E+02
  4878 |   443898 |  1.755096E+02
  4879 |   443989 |  1.755456E+02
  4880 |   444080 |  1.755789E+02
  4881 |   444171 |  1.756229E+02
  4882 |   444262 |  1.756629E+02
  4883 |   444353 |  1.757029E+02
  4884 |   444444 |  1.757389E+02
  4885 |   444535 |  1.757749E+02
  4886 |   444626 |  1.758269E+02
  4887 |   444717 |  1.758629E+02
  4888 |   444808 |  1.759029E+02
  4889 |   444899 |  1.759389E+02
  4890 |   444990 |  1.759749E+02
  4891 |   445081 |  1.760069E+02
  4892 |   445172 |  1.760469E+02
  4893 |   445263 |  1.760749E+02
  4894 |   445354 |  1.761069E+02
  4895 |   445445 |  1.761389E+02
  4896 |   445536 |  1.761709E+02
  4897 |   445627 |  1.762069E+02
  4898 |   445718 |  1.762389E+02
  4899 |   445809 |  1.762709E+02
  4900 |   445900 |  1.763029E+02
  4901 |   445991 |  1.763269E+02
  4902 |   446

In [ ]:
with open('pymoo_pop_nsga3_motsp_91_500_3_100.csv', 'w', newline='') as f, open('pymoo_fitness_nsga3_motsp_91_500_3_100.csv','w', newline='') as g:
    writer = csv.writer(f)    
    writerFitness = csv.writer(g)
    writer.writerow(["f1","f2","f3"])
    
    for fitness in res_motsp_nsga3.F:
            writerFitness.writerow(fitness)
            
    for pop in res_motsp_nsga3.X:
            writer.writerow(pop)

In [18]:
with open('pymoo_pop_nsga2_motsp_92_500_3_100.csv', 'w', newline='') as f, open('pymoo_fitness_nsga2_motsp_92_500_3_100.csv','w', newline='') as g:
    writer = csv.writer(f)    
    writerFitness = csv.writer(g)
    writer.writerow(["f1","f2","f3"])
    
    for fitness in res_motsp_nsga2.F:
            writerFitness.writerow(fitness)
            
    for pop in res_motsp_nsga2.X:
            writer.writerow(pop)